# **P2: Bi-Objective linear optimization problem using abstract pyomo model**

Create the simple abstract model:
$$min \sum^{n}_{j=1} c_{j} x_{j}$$
$$s.t. \sum^{n}_{j=1}a_{ij}x_{j}\leq b_{i} \forall i=1...m$$
$$x_{j}\leq 0 \forall j=1...n $$


In [1]:
import csv
import pandas
import os
import fnmatch
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *

In [4]:
model = AbstractModel()
model.m = Param(within=NonNegativeIntegers)
model.n = Param(within=NonNegativeIntegers)

model.I = RangeSet(1, model.m)
model.J = RangeSet(1, model.n)

model.a = Param(model.I, model.J)
model.b = Param(model.I)
model.c = Param(model.J)

# the next line declares a variable indexed by the set J
model.x = Var(model.J, domain=NonNegativeReals)

def obj_expression(model):
    return summation(model.c, model.x)

model.OBJ = Objective(rule=obj_expression)

def ax_constraint_rule(model, i):
    # return the expression for the constraint for i
    return sum(model.a[i,j] * model.x[j] for j in model.J) <= model.b[i]

# the next line creates one constraint for each member of the set model.I
model.AxbConstraint = Constraint(model.I, rule=ax_constraint_rule)

Use the csv data to initialize the instance

In [75]:
files = os.listdir()
if ('constraints.csv' in files)==False or ('objectives.csv' in files)==False:
    with open("dataCreator.py", mode='r') as dataCreator:
        exec(dataCreator.read())
obj_file = pandas.read_csv('objectives.csv')
c_file = pandas.read_csv('constraints.csv')
A = obj_file.columns.tolist()
A.append('UB')
B = sum(1 for line in obj_file)
model.n = sum(1 for column in obj_file) 
model.m = sum(1 for line in c_file)

data = DataPortal()
data.load(filename='constraints.csv', select=(A), param=(model.a,model.b))
A.remove('UB')
c=[]
for z in np.linspace(0,1,10):
    w=[z,1-z]
    for i in A: 
        for j in range(B):
            c = obj_file[i][j]*w[j] 
print(c)

0.0
